In [ ]:
!pip -q install langchain openai huggingface_hub

In [3]:
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ""

# Plain Conditional Generation

## Using OpenAI GPT3

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name = 'text-davinci-003',
             temperature = 0.5,
             max_tokens = 256)


In [28]:
text = "Why did the chicken cross the road?"
print(llm(text))

it was thirsty


## Now using T5-Flan-XL

In [4]:
from langchain.llms import HuggingFaceHub
llm_hf = HuggingFaceHub(repo_id = "google/flan-t5-xxl", model_kwargs = {"temperature": 0.9, "max_length":512})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [32]:
text = "Why did the chicken cross the road?"
print(llm_hf(text, ))

it wanted to get to the other side


# Prompt Templates

In [49]:
from langchain import PromptTemplate

restaurant_template = """
I want you to act as a naming consultant for new restaurants.

Return a list of 5 restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.

What are some good names for a restaurant that is {restaurant_description}?
"""

prompt = PromptTemplate(
    input_variables = ['restaurant_description'],
    template = restaurant_template
)

In [63]:
desc_1 = "A Indian Paneer cousin restaurant with musical fountain"
desc_2 = "A Mallaio shop in the spiritual capital of India having ganga view"
desc_3 = "a cafe that has live hard rock music and memorabilia"

In [64]:
prompt.format(restaurant_description = desc_1)


'\nI want you to act as a naming consultant for new restaurants.\n\nReturn a list of 5 restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.\n\nWhat are some good names for a restaurant that is A Indian Paneer cousin restaurant with musical fountain?\n'

In [65]:
## querying the model with the prompt template
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run(desc_1))

Spice Island


## With Few Shot Learning

In [5]:
from langchain import PromptTemplate, FewShotPromptTemplate

# First, create the list of few shot examples.
examples = [
    {
        "word": "happy",
        "antonym": "sad"
        },
    {
        "word": "tall",
        "antonym": "short"
        },
]

In [6]:
# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

In [7]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n\n",
)


In [8]:

# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: big
Antonym:


In [10]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm_hf, prompt=few_shot_prompt)

# Run the chain only specifying the input variable.
print(chain.run("Big"))

small


In [11]:
print(chain.run(input = "Largest"))

smallest


In [12]:
print(chain.run("Rough"))

smooth


In [13]:
print(chain.run("Dark"))

Light


In [14]:
print(chain.run("Floor"))

Ceiling
